# V7 Quantum Training - Colab via VS Code

Bu notebook VS Code'daki Google Colab eklentisi ile calistirilmak uzere hazirlanmistir.

**Adimlar:**
1. Kernel olarak "Colab" secin (A100 GPU)
2. Hucreleri sirayla calistirin
3. Training ~5-6 saat surecek (3 epoch)

## 1. GPU Kontrolu

In [ ]:
!nvidia-smi
import torch
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")

## 2. Google Drive Mount + Repo Clone

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

REPO_DIR = '/content/Quanvolutional-Neural-Network'

if os.path.exists(REPO_DIR):
    !cd {REPO_DIR} && git pull
    print("Repo guncellendi.")
else:
    !git clone https://github.com/necatiincekara/Quanvolutional-Neural-Network.git {REPO_DIR}
    print("Repo klonlandi.")

os.chdir(REPO_DIR)
print(f"Calisma dizini: {os.getcwd()}")

## 3. Dependencies

In [ ]:
!pip install pennylane pennylane-lightning-gpu "numpy>=2.0" tqdm matplotlib -q

import pennylane as qml
print(f"PennyLane: {qml.__version__}")
print(f"PyTorch: {torch.__version__}")

## 4. Ortam Dogrulama

In [ ]:
from src import config

print(f"Platform:  Colab={config.IS_COLAB}")
print(f"Compute:   {config.DEVICE}")
print(f"Quantum:   {config.QUANTUM_DEVICE}")
print(f"Train:     {config.TRAIN_PATH} (exists: {os.path.exists(config.TRAIN_PATH)})")
print(f"Test:      {config.TEST_PATH} (exists: {os.path.exists(config.TEST_PATH)})")

assert os.path.exists(config.TRAIN_PATH), f"HATA: {config.TRAIN_PATH} bulunamadi! Drive mount edildi mi?"
assert config.DEVICE == 'cuda', "HATA: GPU bulunamadi! Runtime tipini kontrol edin."
print("\nHer sey hazir!")

## 5. Model Test (Hizli kontrol)

In [ ]:
from src.trainable_quantum_model import create_enhanced_model

model = create_enhanced_model(circuit_type='data_reuploading', num_classes=44)
total = sum(p.numel() for p in model.parameters())
q = sum(p.numel() for n, p in model.named_parameters() if 'quanv' in n)
print(f"V7 Model: {total} params (quantum: {q}, classical: {total-q})")

# Hizli forward pass testi
x = torch.randn(2, 1, 32, 32).to(config.DEVICE)
model = model.to(config.DEVICE)
with torch.no_grad():
    out = model(x)
print(f"Forward pass: {x.shape} -> {out.shape}")
print("Model GPU'da calisiyor!")

## 6. V7 TRAINING BASLAT

**Tahmini sure: ~5-6 saat (3 epoch)**

### Saglikli cikti gormek istediginiz seyler:
- `quantum grad mean` = 1e-3 ~ 1e-1 arasi
- `classical grad mean` = 1e-2 ~ 1e+0 arasi
- Loss'un ilk 10 batch'te dusmesi
- Accuracy'nin 0%'den yukari cikmasi (ilk epoch sonunda)

### Alarm isaretleri (durdurmaniz gerekir):
- `quantum grad mean=0.00e+00` (gradient oldu)
- 1 epoch bittikten sonra hala `acc=0.00%`
- `CUDA out of memory`

### Durdurmak icin:
- Hucrenin solundaki **kare (stop) butonu**
- Ya da `Interrupt Kernel` (Command Palette)

In [ ]:
from src.enhanced_training import run_enhanced_training
import time

start = time.time()

best_acc, test_acc = run_enhanced_training(
    circuit_type='data_reuploading',
    num_epochs=3
)

elapsed = time.time() - start
hours = int(elapsed // 3600)
minutes = int((elapsed % 3600) // 60)
print(f"\nToplam sure: {hours}s {minutes}dk")
print(f"En iyi val accuracy: {best_acc:.2f}%")
print(f"Test accuracy: {test_acc:.2f}%")

## 7. Sonuclari Kaydet

In [ ]:
import shutil

# Checkpoint'lari Drive'a kopyala
drive_backup = '/content/drive/MyDrive/quanv_results/v7/'
os.makedirs(drive_backup, exist_ok=True)

# En iyi modeli kaydet
if os.path.exists('models/best_v7_model.pth'):
    shutil.copy('models/best_v7_model.pth', drive_backup)
    print(f"Model kaydedildi: {drive_backup}best_v7_model.pth")

# Experiment dizinini kaydet
import glob
exp_dirs = glob.glob('experiments/v7_*')
if exp_dirs:
    latest_exp = max(exp_dirs, key=os.path.getmtime)
    dest = os.path.join(drive_backup, os.path.basename(latest_exp))
    if os.path.exists(dest):
        shutil.rmtree(dest)
    shutil.copytree(latest_exp, dest)
    print(f"Experiment kaydedildi: {dest}")

print(f"\nTum sonuclar Drive'a kaydedildi: {drive_backup}")